In [1]:
from ast import literal_eval
import json
import pandas as pd
import numpy as np

In [2]:
#All local crossword related files
INPUT_PUZZLE_CSV = "onepuzzle.csv"
INPUT_FAISS_HINTS_CSV = "Ianswers.csv"

In [3]:
#Input crossword puzzle in raw format
input_crossword = pd.read_csv(INPUT_PUZZLE_CSV)
input_crossword.head()

,matrix,direction,clue_number,clue,answer,date
0,[[1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1....,"['ACROSS', 'ACROSS', 'ACROSS', 'ACROSS', 'ACRO...","[1, 9, 13, 15, 16, 18, 19, 20, 22, 23, 24, 25,...","['Eye exam you need to pass?', 'Soaks (up)', ""...","['IRISSCAN', 'SOPS', 'CENTURIES', 'EPEE', 'ESC...",12/10/2022


In [4]:
#Output of FAISS index search
faiss_answers = pd.read_csv(INPUT_FAISS_HINTS_CSV)
faiss_answers.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,IGY,RCAF,SWC,ABM,ABM,ABM,ABM,ABM,ABM,ABM,...,ENV,ENV,ENV,ENV,RELS,RELS,RELS,RELS,RELS,RELS
1,TARTU,TULA,OULU,OULU,KALAMATA,ESPOO,ESPOO,ESPOO,LAPERI,LAA,...,O,O,O,O,O,O,O,O,O,O
2,TAXCO,TAXCO,TAXCO,TAXCO,TAXCO,TAXCO,TAXCO,TAXCO,ORURO,YSIDRO,...,CINCO,CINCO,CINCO,CINCO,CINCO,CINCO,CINCO,CINCO,CINCO,CINCO
3,TAKETH,TAKETH,TAKETH,RULETH,GOETH,GOETH,GOETH,EATETH,ORME,ORME,...,ILET,ACTOF,ACTOF,THERENT,THERENT,THERENT,ATALE,ATALE,ATALE,ATALE
4,L,L,L,L,L,L,L,L,L,L,...,SOMALIS,SOMALIS,SOMALIS,FATAH,FATAH,FATAH,FATAH,FATAH,FATAH,FATAH


In [5]:
# Extract clues
input_crossword_clue = input_crossword.loc[0]['clue'] \
    .replace("[","") \
    .replace("\']","{end}") \
    .replace("]","") \
    .replace("\',","{end}") \
    .replace("\",","{end}") \
    .replace("\'s ","{apos}") \
    .replace("\'","{start}") \
    .replace("{end} \"","{end} {start}") \
    .replace("{end} {start}","||||||") \
    .replace("{start}","") \
    .replace("{end}","").split("||||||")
input_crossword_clue[0:10]

['Eye exam you need to pass?',
 'Soaks (up)',
 'Cricketer{apos}100-run streaks',
 'Weapon with a point darrêt',
 'Where making a hasty exit is encouraged',
 'Flour ground in a chakki',
 'Pumped',
 'The Father of ___, moniker for inventor Leo Baekeland',
 'Not as adventurous',
 'Game that helps teach object permanence']

In [6]:
# Extract answers
input_crossword_answer = input_crossword.loc[0]['answer'] \
    .replace("[","") \
    .replace("]","") \
    .replace("'","") \
    .replace(" ","") \
    .split(",")
input_crossword_answer[0:10]

['IRISSCAN',
 'SOPS',
 'CENTURIES',
 'EPEE',
 'ESCAPEROOM',
 'ATTA',
 'STOKED',
 'PLASTICS',
 'TAMER',
 'PEEKABOO']

In [8]:
#Extract the crossword in the matrix form

def get_rawcrossword_row(raw_row, x, y):
    row = []
    for i in range(len(raw_row)):
        if raw_row[i] == x:
            row.append(1)
        elif raw_row[i] == y:
            row.append(0)
    return row

input_crossword_matrix = input_crossword.loc[0]['matrix'] \
    .replace("[[","[") \
    .replace("]]","]") \
    .replace("1. ","1,") \
    .replace("0. ","0,") \
    .replace(".","") \
    .split("\n ")

# input_crossword_matrix
input_crossword_matrix = [get_rawcrossword_row(i,"1","0") for i in input_crossword_matrix]
input_crossword_matrix

[[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1],
 [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]

In [12]:
# Extract Across direction Row/Col start locations

clue_across = []
for row in range(input_crossword_matrix.shape[0]):
    begin = 0
    for col in range(input_crossword_matrix.shape[1]):
        if begin == 0 and input_crossword_matrix[row][col] == 1:
            clue_across.append((row,col))
            begin = 1
        elif input_crossword_matrix[row][col] == 0:
            begin = 0

clue_across[0:10]

[(0, 0),
 (0, 11),
 (1, 0),
 (1, 11),
 (2, 0),
 (2, 11),
 (3, 0),
 (3, 7),
 (4, 0),
 (4, 6)]

In [13]:
# Extract Across direction Row/Col start locations 

clue_down = []
pointer = np.zeros(input_crossword_matrix.shape[1])
for row in range(input_crossword_matrix.shape[0]):
    for col in range(input_crossword_matrix.shape[1]):
        if input_crossword_matrix[row][col] == 1 and pointer[col] == 0:
            clue_down.append((row,col))
            pointer[col] = 1
        elif input_crossword_matrix[row][col] == 0:
            pointer[col] = 0
            

clue_down[0:10]

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 11),
 (0, 12)]

In [16]:
# Merge both Across and Down directions

input_crossword_clue_RC = clue_across + clue_down
input_crossword_clue_RC[0:10]

[(0, 0),
 (0, 11),
 (1, 0),
 (1, 11),
 (2, 0),
 (2, 11),
 (3, 0),
 (3, 7),
 (4, 0),
 (4, 6)]

In [17]:
# Extract Across and Down indicator from the raw crossword

input_crossword_direction = input_crossword.loc[0]['direction'] \
    .replace("[","") \
    .replace("]","") \
    .replace(", ",",") \
    .replace(" ","") \
    .replace("'ACROSS'","A") \
    .replace("'DOWN'","D") \
    .split(",")

input_crossword_direction[0:10]

['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']

In [18]:
# Print and verify Crossword Summary
input_crossword_matrix = np.array(input_crossword_matrix)

print(f"Crossword size: {input_crossword_matrix.shape}")
print(f"Number of clues: {len(input_crossword_clue)}")
print(f"Number of answers: {len(input_crossword_answer)}")
print(f"Number of directions: {len(input_crossword_direction)}")
print(f"Number of directions RCs: {len(input_crossword_clue_RC)}")

Crossword size: (15, 15)
Number of clues: 64
Number of answers: 64
Number of directions: 64
Number of directions RCs: 64


In [19]:
# Visualize the extracted Crossword data

puzzle = []
for i in range(len(input_crossword_clue)):
    puzzle.append((input_crossword_clue[i], input_crossword_clue_RC[i], input_crossword_direction[i], input_crossword_answer[i]))
    
# puzzle as dataframe
puzzle_pd = pd.DataFrame(puzzle, columns=['clue','RC','direction','answer'])
puzzle_pd

,clue,RC,direction,answer
0,Eye exam you need to pass?,"(0, 0)",A,IRISSCAN
1,Soaks (up),"(0, 11)",A,SOPS
2,Cricketer{apos}100-run streaks,"(1, 0)",A,CENTURIES
3,Weapon with a point darrêt,"(1, 11)",A,EPEE
4,Where making a hasty exit is encouraged,"(2, 0)",A,ESCAPEROOM
...,...,...,...,...
59,"Jack rabbits, but not rabbits","(10, 1)",D,HARES
60,Part of an epic verse,"(10, 10)",D,CANTO
61,Bhikkhu{apos}teacher,"(11, 0)",D,LAMA
62,Set askew,"(11, 9)",D,TILT


In [37]:
# Format the crossword for loading to graphDb

crossword_to_solve = {}

for row in puzzle_pd.itertuples(index=True, name='Pandas'):
    crossword_to_solve[row.clue] = {"start": row.RC , "direction": row.direction, "answer": row.answer, "length": len(row.answer)}

crossword_to_solve

{'Eye exam you need to pass?': {'start': (0, 0),
  'direction': 'A',
  'answer': 'IRISSCAN',
  'length': 8},
 'Soaks (up)': {'start': (0, 11),
  'direction': 'A',
  'answer': 'SOPS',
  'length': 4},
 'Cricketer{apos}100-run streaks': {'start': (1, 0),
  'direction': 'A',
  'answer': 'CENTURIES',
  'length': 9},
 'Weapon with a point darrêt': {'start': (1, 11),
  'direction': 'A',
  'answer': 'EPEE',
  'length': 4},
 'Where making a hasty exit is encouraged': {'start': (2, 0),
  'direction': 'A',
  'answer': 'ESCAPEROOM',
  'length': 10},
 'Flour ground in a chakki': {'start': (2, 11),
  'direction': 'A',
  'answer': 'ATTA',
  'length': 4},
 'Pumped': {'start': (3, 0),
  'direction': 'A',
  'answer': 'STOKED',
  'length': 6},
 'The Father of ___, moniker for inventor Leo Baekeland': {'start': (3, 7),
  'direction': 'A',
  'answer': 'PLASTICS',
  'length': 8},
 'Not as adventurous': {'start': (4, 0),
  'direction': 'A',
  'answer': 'TAMER',
  'length': 5},
 'Game that helps teach object 

In [40]:
# Format the FAISS hints for loading to graphDb 

# crossword_hints = {}
# i = 0
# for key in crossword_to_solve.keys():
#     clues = np.array(faiss_answers.iloc[i:i+1,:])
#     crossword_hints[key] = list(set(clues[0]))[0:25] 
#     i = i + 1
    
# crossword_hints    
crossword_hints = {}
i = 0
master = {}
for key in crossword_to_solve.keys():
    clues = np.array(faiss_answers.iloc[i:i+1,:])  
    master = {}
    for k in range(len(clues[0])):
        if clues[0][k] not in master: #and len(clues[0][k]) == crossword_to_solve[key]['length']:
             master[clues[0][k]] = k
    
    #master[crossword_to_solve[key]['answer']] =  0 ## hack to validate crossword by injecting true answer
    crossword_hints[key] = master
    i = i + 1
    
crossword_hints  

{'Eye exam you need to pass?': {'IGY': 0,
  'RCAF': 1,
  'SWC': 2,
  'ABM': 3,
  'SWA': 15,
  'RFC': 16,
  'NLE': 17,
  'SNR': 21,
  'ORL': 22,
  'SWED': 27,
  'SWM': 31,
  'AHL': 41,
  'USLTA': 51,
  'UMW': 56,
  'LDR': 63,
  'ANDL': 67,
  'IFC': 68,
  'DMV': 71,
  'FRS': 82,
  'ENCYC': 84,
  'SANDM': 89,
  'THENHL': 92,
  'THEBBC': 97,
  'THENFL': 99,
  'SCM': 100,
  'ASC': 101,
  'BSC': 102,
  'ENCS': 104,
  'DMVS': 110,
  'ADL': 114,
  'SANDL': 116,
  'INITS': 123,
  'INSC': 142,
  'USP': 143,
  'APOC': 144,
  'IHS': 149,
  'SANDP': 153,
  'USAR': 162,
  'LRG': 164,
  'INV': 165,
  'SWF': 173,
  'BKS': 184,
  'SAE': 188,
  'MSGR': 219,
  'EXCH': 232,
  'ENCL': 234,
  'ATC': 259,
  'EXC': 264,
  'MLLE': 268,
  'AGN': 295,
  'ECCL': 297,
  'AFS': 304,
  'SRTAS': 305,
  'SLC': 335,
  'TCHR': 340,
  'JRR': 343,
  'NCC': 344,
  'IOC': 345,
  'SM': 375,
  'ISLS': 376,
  'NGS': 382,
  'AAM': 383,
  'USTA': 384,
  'EDH': 412,
  'SRA': 424,
  'HNC': 580,
  'RAF': 581,
  'NASL': 652,
  'ENC'

In [41]:
# Save the crossword and hints as JSON
with open("crossword_to_solve.json", "w") as outfile:
    json.dump(literal_eval(str(crossword_to_solve)), outfile)

with open("crossword_hints.json", "w") as outfile:
    json.dump(literal_eval(str(crossword_hints)), outfile)